<a href="https://colab.research.google.com/github/jbishop45/CS-7650/blob/project-2/project_2_NER_release_sp23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# Licensing Information:  You are free to use or extend this project for
# educational purposes provided that (1) you do not distribute or publish
# solutions, (2) you retain this notice, and (3) you provide clear
# attribution to The Georgia Institute of Technology, including a link to  https://aritter.github.io/CS-7650-sp22/

# Attribution Information: This assignment was developed at The Georgia Institute of Technology
# by Alan Ritter (alan.ritter@cc.gatech.edu)
# Contributors: Xurui Zhang (Spring 2022)

# Project #2: Named Entity Recognition

In this assignment, you will implement a bidirectional LSTM-CNN-CRF for sequence labeling, following [this paper by Xuezhe Ma and Ed Hovy](https://www.aclweb.org/anthology/P16-1101.pdf), on the CoNLL named entity recognition dataset.  Before starting the assignment, we recommend reading the Ma and Hovy paper.

First, let's import some libraries and make sure the runtime has access to a GPU.


In [14]:
import torch
import torch.nn as nn
import torch.optim as optim

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
    print('and then re-execute this cell.')
else:
    print(gpu_info)

print(f'GPU available: {torch.cuda.is_available()}')

Thu Mar 23 18:56:54 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Download the Data

Run the following code to download the English part of the CoNLL 2003 dataset, the evaluation script and pre-filtered GloVe embeddings we are providing for this data.

In [15]:
#CoNLL 2003 data
!wget https://raw.githubusercontent.com/patverga/torch-ner-nlp-from-scratch/master/data/conll2003/eng.train
!wget https://raw.githubusercontent.com/patverga/torch-ner-nlp-from-scratch/master/data/conll2003/eng.testa
!wget https://raw.githubusercontent.com/patverga/torch-ner-nlp-from-scratch/master/data/conll2003/eng.testb
!cat eng.train | awk '{print $1 "\t" $4}' > train
!cat eng.testa | awk '{print $1 "\t" $4}' > dev
!cat eng.testb | awk '{print $1 "\t" $4}' > test

#Evaluation Script
!wget https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl

#Pre-filtered GloVe embeddings
!wget https://raw.githubusercontent.com/aritter/aritter.github.io/master/files/glove.840B.300d.conll_filtered.txt

--2023-03-23 18:56:54--  https://raw.githubusercontent.com/patverga/torch-ner-nlp-from-scratch/master/data/conll2003/eng.train
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3283420 (3.1M) [text/plain]
Saving to: ‘eng.train.1’

eng.train.1         100%[===================>]   3.13M  --.-KB/s    in 0.07s   

2023-03-23 18:56:54 (44.6 MB/s) - ‘eng.train.1’ saved [3283420/3283420]

--2023-03-23 18:56:54--  https://raw.githubusercontent.com/patverga/torch-ner-nlp-from-scratch/master/data/conll2003/eng.testa
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting respo

## CoNLL Data Format

Run the following cell to see a sample of the data in CoNLL format.  As you can see, each line in the file represents a word and its labeled named entity tag in BIO format.  A blank line is used to seperate sentences.

In [16]:
!head -n 20 train

-DOCSTART-	O
	
EU	I-ORG
rejects	O
German	I-MISC
call	O
to	O
boycott	O
British	I-MISC
lamb	O
.	O
	
Peter	I-PER
Blackburn	I-PER
	
BRUSSELS	I-LOC
1996-08-22	O
	
The	O
European	I-ORG


## Reading in the Data

Below we proivide a bit of code to read in data in the CoNLL format.  This also reads in the filtered GloVe embeddings, to save you some effort - we will discuss this more later.

In [17]:
#Read in the training data
def read_conll_format(filename):
    (words, tags, currentSent, currentTags) = ([],[],['-START-'],['START'])
    for line in open(filename).readlines():
        line = line.strip()
        #print(line)
        if line == "":
            currentSent.append('-END-')
            currentTags.append('END')
            words.append(currentSent)
            tags.append(currentTags)
            (currentSent, currentTags) = (['-START-'], ['START'])
        else:
            (word, tag) = line.split()
            currentSent.append(word)
            currentTags.append(tag)
    return (words, tags)

def sentences2char(sentences):
    return [[['start'] + [c for c in w] + ['end'] for w in l] for l in sentences]


(sentences_train, tags_train) = read_conll_format("train")
(sentences_dev, tags_dev)     = read_conll_format("dev")

print(sentences_train[2])
print(tags_train[2])

sentencesChar = sentences2char(sentences_train)

print(sentencesChar[2])

['-START-', 'Peter', 'Blackburn', '-END-']
['START', 'I-PER', 'I-PER', 'END']
[['start', '-', 'S', 'T', 'A', 'R', 'T', '-', 'end'], ['start', 'P', 'e', 't', 'e', 'r', 'end'], ['start', 'B', 'l', 'a', 'c', 'k', 'b', 'u', 'r', 'n', 'end'], ['start', '-', 'E', 'N', 'D', '-', 'end']]


In [18]:
#Read GloVe embeddings.
def read_GloVe(filename):
    embeddings = {}
    for line in open(filename).readlines():
        #print(line)
        fields = line.strip().split(" ")
        word = fields[0]
        embeddings[word] = [float(x) for x in fields[1:]]
    return embeddings

GloVe = read_GloVe("glove.840B.300d.conll_filtered.txt")

print(GloVe["the"])
print("dimension of glove embedding:", len(GloVe["the"]))

[0.27204, -0.06203, -0.1884, 0.023225, -0.018158, 0.0067192, -0.13877, 0.17708, 0.17709, 2.5882, -0.35179, -0.17312, 0.43285, -0.10708, 0.15006, -0.19982, -0.19093, 1.1871, -0.16207, -0.23538, 0.003664, -0.19156, -0.085662, 0.039199, -0.066449, -0.04209, -0.19122, 0.011679, -0.37138, 0.21886, 0.0011423, 0.4319, -0.14205, 0.38059, 0.30654, 0.020167, -0.18316, -0.0065186, -0.0080549, -0.12063, 0.027507, 0.29839, -0.22896, -0.22882, 0.14671, -0.076301, -0.1268, -0.0066651, -0.052795, 0.14258, 0.1561, 0.05551, -0.16149, 0.09629, -0.076533, -0.049971, -0.010195, -0.047641, -0.16679, -0.2394, 0.0050141, -0.049175, 0.013338, 0.41923, -0.10104, 0.015111, -0.077706, -0.13471, 0.119, 0.10802, 0.21061, -0.051904, 0.18527, 0.17856, 0.041293, -0.014385, -0.082567, -0.035483, -0.076173, -0.045367, 0.089281, 0.33672, -0.22099, -0.0067275, 0.23983, -0.23147, -0.88592, 0.091297, -0.012123, 0.013233, -0.25799, -0.02972, 0.016754, 0.01369, 0.32377, 0.039546, 0.042114, -0.088243, 0.30318, 0.087747, 0.1634

## Mapping Tokens to Indices

As in the last project, we will need to convert words in the dataset to numeric indices, so they can be presented as input to a neural network.  Code to handle this for you with sample usage is provided below.

In [19]:
#Create mappings between tokens and indices.

from collections import Counter
import random

#Will need this later to remove 50% of words that only appear once in the training data from the vocabulary (and don't have GloVe embeddings).
wordCounts = Counter([w for l in sentences_train for w in l])
charCounts = Counter([c for l in sentences_train for w in l for c in w])
singletons = set([w for (w,c) in wordCounts.items() if c == 1 and not w in GloVe.keys()])
charSingletons = set([w for (w,c) in charCounts.items() if c == 1])

#Build dictionaries to map from words, characters to indices and vice versa.
#Save first two words in the vocabulary for padding and "UNK" token.
word2i = {w:i+2 for i,w in enumerate(set([w for l in sentences_train for w in l] + list(GloVe.keys())))}
char2i = {w:i+2 for i,w in enumerate(set([c for l in sentencesChar for w in l for c in w]))}
i2word = {i:w for w,i in word2i.items()}
i2char = {i:w for w,i in char2i.items()}

vocab_size = max(word2i.values()) + 1
char_vocab_size = max(char2i.values()) + 1

#Tag dictionaries.
tag2i = {w:i for i,w in enumerate(set([t for l in tags_train for t in l]))}
i2tag = {i:t for t,i in tag2i.items()}

#When training, randomly replace singletons with UNK tokens sometimes to simulate situation at test time.
def getDictionaryRandomUnk(w, dictionary, train=False):
    if train and (w in singletons and random.random() > 0.5):
        return 1
    else:
        return dictionary.get(w, 1)

#Map a list of sentences from words to indices.
def sentences2indices(words, dictionary, train=False):
    #1.0 => UNK
    return [[getDictionaryRandomUnk(w,dictionary, train=train) for w in l] for l in words]

#Map a list of sentences containing to indices (character indices)
def sentences2indicesChar(chars, dictionary):
    #1.0 => UNK
    return [[[dictionary.get(c,1) for c in w] for w in l] for l in chars]

#Indices
X       = sentences2indices(sentences_train, word2i, train=True)
X_char  = sentences2indicesChar(sentencesChar, char2i)
Y       = sentences2indices(tags_train, tag2i)

print("vocab size:", vocab_size)
print("char vocab size:", char_vocab_size)
print()

print("index of word 'the':", word2i["the"])
print("word of index 253:", i2word[253])
print()

#Print out some examples of what the dev inputs will look like
for i in range(10):
    print(" ".join([i2word.get(w,'UNK') for w in X[i]]))

vocab size: 29148
char vocab size: 88

index of word 'the': 28319
word of index 253: Schalke

-START- -DOCSTART- -END-
-START- EU rejects German call to boycott British lamb . -END-
-START- Peter Blackburn -END-
-START- BRUSSELS 1996-08-22 -END-
-START- The European Commission said on Thursday it disagreed with German advice to consumers to shun British lamb until scientists determine whether mad cow disease can be transmitted to sheep . -END-
-START- Germany 's representative to the European Union 's veterinary committee Werner Zwingmann said on Wednesday consumers should buy sheepmeat from countries other than Britain until the scientific advice was clearer . -END-
-START- " We do n't support any such recommendation because we do n't see any grounds for it , " the Commission 's chief spokesman Nikolaus van der Pas told a news briefing . -END-
-START- He said further scientific study was required and if it was found that action was needed it should be taken by the European Union . -EN

In [20]:
[print(i,i2word[i],word2i[i2word[i]]) for i in range(2,10)]

2 boxed 2
3 Brechin 3
4 softs 4
5 Afrikaans 5
6 thinks 6
7 balls 7
8 Igor 8
9 Bollore 9


[None, None, None, None, None, None, None, None]

## Padding and Batching

In this assignment, you should train your models using minibatched SGD, rather than using a batch size of 1 as we did in the previous project.  When presenting multiple sentences to the network at the same time, we will need to pad them to be of the same length. We use [torch.nn.utils.rnn.pad_sequence](https://pytorch.org/docs/stable/generated/torch.nn.utils.rnn.pad_sequence.html) to do so.

Below we provide some code to prepare batches of data to present to the network.  pad the sequence so that all the sequences have the same length.

**Side Note:** PyTorch includes utilities in [`torch.utils.data`](https://pytorch.org/docs/stable/data.html) to help with padding, batching, shuffling and some other things, but for this assignment we will do everything from scratch to help you see exactly how this works.

In [21]:
#Pad inputs to max sequence length (for batching)
def prepare_input(X_list):
    X_padded = torch.nn.utils.rnn.pad_sequence([torch.as_tensor(l) for l in X_list], batch_first=True).type(torch.LongTensor) # padding the sequences with 0
    X_mask   = torch.nn.utils.rnn.pad_sequence([torch.as_tensor([1.0] * len(l)) for l in X_list], batch_first=True).type(torch.FloatTensor) # consisting of 0 and 1, 0 for padded positions, 1 for non-padded positions
    return (X_padded, X_mask)

#Maximum word length (for character representations)
MAX_CLEN=32

def prepare_input_char(X_list):
    MAX_SLEN = max([len(l) for l in X_list])
    X_padded  = [l + [[]]*(MAX_SLEN-len(l))  for l in X_list]
    X_padded  = [[w[0:MAX_CLEN] for w in l] for l in X_padded]
    X_padded  = [[w + [1]*(MAX_CLEN-len(w)) for w in l] for l in X_padded]
    return torch.as_tensor(X_padded).type(torch.LongTensor)

#Pad outputs using one-hot encoding
def prepare_output_onehot(Y_list, NUM_TAGS=max(tag2i.values())+1):
    Y_onehot = [torch.zeros(len(l), NUM_TAGS) for l in Y_list]
    for i in range(len(Y_list)):
        for j in range(len(Y_list[i])):
            Y_onehot[i][j,Y_list[i][j]] = 1.0
    Y_padded = torch.nn.utils.rnn.pad_sequence(Y_onehot, batch_first=True).type(torch.FloatTensor)
    return Y_padded

print("max slen:", max([len(x) for x in X_char]))  #Max sequence length in the training data is 39.

(X_padded, X_mask) = prepare_input(X)
X_padded_char      = prepare_input_char(X_char)
Y_onehot           = prepare_output_onehot(Y)

print("X_padded:", X_padded.shape)
print("X_mask:", X_mask.shape)
print("X_padded_char:", X_padded_char.shape)
print("Y_onehot:", Y_onehot.shape)

max slen: 115
X_padded: torch.Size([14987, 115])
X_mask: torch.Size([14987, 115])
X_padded_char: torch.Size([14987, 115, 32])
Y_onehot: torch.Size([14987, 115, 10])


In [22]:
input = prepare_input(X[0:5])[0]
print("input: ", input.shape)

input:  torch.Size([5, 32])


## **Your code starts here:** Basic LSTM Tagger (10 points)

OK, now you should have everything you need to get started.

Recall that your goal is to to implement the BiLSTM-CNN-CRF, as described in [(Ma and Hovy, 2016)](https://www.aclweb.org/anthology/P16-1101.pdf).  This is a relatively complex network with various components.  Below we provide starter code to break down your implementation into increasingly complex versions of the final model, starting with a Basic LSTM tagger.  This way you can be confident that each part is working correctly before incrementally increasing the complexity of your implementation.  This is generally a good approach to take when implementing complex models, since buggy PyTorch code is often partially working, but produces worse results than a correct implementation, so it's hard to know whether added complexities are helping or hurting.  Also, if you aren't able to match published results it's hard to know which component of your model has the problem (or even whether or not it is a problem in the published result!)

Fill in the functions marked as `TODO` in the code block below.  If everything is working correctly, you should be able to achieve an **F1 score of 0.87 on the dev set and 0.83 on the test set (with GloVe embeddings)**. You are required to initialize word embeddings with GloVe later, but you can randomly initialize the word embeddings in the beginning.

In [104]:
class BasicLSTMtagger(nn.Module):
    def __init__(self, DIM_EMB=10, DIM_HID=10, VOCAB_SIZE=29148, debug=False):
        super(BasicLSTMtagger, self).__init__()
        NUM_TAGS = max(tag2i.values())+1
        (self.DIM_EMB, self.NUM_TAGS) = (DIM_EMB, NUM_TAGS)

        #TODO: initialize parameters - embedding layer, nn.LSTM, nn.Linear and nn.LogSoftmax
        bidirectional=True
        in_features = DIM_HID*2 if bidirectional else DIM_HID

        self.word_embeddings = nn.Embedding(num_embeddings=VOCAB_SIZE,embedding_dim=DIM_EMB)
        self.lstm = nn.LSTM(input_size=DIM_EMB, hidden_size=DIM_HID, batch_first=True, bidirectional=bidirectional)
        self.hidden2tag = nn.Linear(in_features=in_features, out_features=NUM_TAGS)

        self.debug = debug
        if self.debug:
          print('VOCAB_SIZE: ' + str(VOCAB_SIZE))
          print('NUM_TAGS: '  +str(NUM_TAGS))
          print('DIM_EMB: ' + str(DIM_EMB))
          print('DIM_HID: ' + str(DIM_HID))
          print('bidirectional?: ' + str(bidirectional) + '\n')

    def forward(self, embeds, train=False):
        # X is X_padded from prepare_input()
        #TODO: Implement the forward computation.
        # embeds = self.word_embeddings(X)
        # if self.debug:
        #   print('embeds: ' + str(embeds.shape))

        lstm_out,_ = self.lstm(embeds)
        if self.debug:
          print('lstm_out: ' + str(lstm_out.shape))

        tag_space = self.hidden2tag(lstm_out)
        if self.debug:
          print('tag_space: ' + str(tag_space.shape))

        tag_scores = torch.nn.functional.log_softmax(tag_space,dim=2)
        if self.debug:
          print('tag_scores: ' + str(tag_scores.shape) + '\n')

        return tag_scores
        #return torch.randn((X.shape[0], X.shape[1], self.NUM_TAGS))  #Random baseline.

    def init_glove(self, GloVe):
        #TODO: initialize word embeddings using GloVe (you can skip this part in your first version, if you want, see instructions below).
        for i in range(2,self.word_embeddings.num_embeddings):
          word = i2word[i]
          if word not in singletons:
            try: 
              self.word_embeddings.weight[i,:] = torch.tensor(GloVe[word])
            except KeyError:
              if self.debug:
                print(i,word,'neither in singletons nor GloVe')
        pass

    def inference(self, sentences):
        X       = prepare_input(sentences2indices(sentences, word2i))[0].cuda()
        embed = self.word_embeddings(X)
        pred = self.forward(embed).argmax(dim=2)
        return [[i2tag[pred[i,j].item()] for j in range(len(sentences[i]))] for i in range(len(sentences))]

    def print_predictions(self, words, tags):
        Y_pred = self.inference(words)
        for i in range(len(words)):
            print("----------------------------")
            print(" ".join([f"{words[i][j]}/{Y_pred[i][j]}/{tags[i][j]}" for j in range(len(words[i]))]))
            print("Predicted:\t", Y_pred[i])
            print("Gold:\t\t", tags[i])

    def write_predictions(self, sentences, outFile):
        fOut = open(outFile, 'w')
        for s in sentences:
            y = self.inference([s])[0]
            #print("\n".join(y[1:len(y)-1]))
            fOut.write("\n".join(y[1:len(y)-1]))  #Skip start and end tokens
            fOut.write("\n\n")

#The following code will initialize a model and test that your forward computation runs without errors.
lstm_test   = BasicLSTMtagger(DIM_HID=7, DIM_EMB=300)
word_embeds = lstm_test.word_embeddings(prepare_input(X[0:5])[0])
lstm_output = lstm_test.forward(word_embeds) # torch.Size([5, 32])
Y_onehot    = prepare_output_onehot(Y[0:5])

#Check the shape of the lstm_output and one-hot label tensors.
print("lstm output shape:", lstm_output.shape)
print("Y onehot shape:", Y_onehot.shape, "\n")

lstm output shape: torch.Size([5, 32, 10])
Y onehot shape: torch.Size([5, 32, 10]) 



In [105]:
#Read in the data

(sentences_dev, tags_dev)     = read_conll_format('dev')
(sentences_train, tags_train) = read_conll_format('train')
(sentences_test, tags_test)   = read_conll_format('test')

# Train your Model (10 points)

Next, implement the function below to train your basic BiLSTM tagger.  See [torch.nn.lstm](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html).  Make sure to save your predictions on the test set (`test_pred_lstm.txt`) for submission to GradeScope. Feel free to change number of epochs, optimizer, learning rate and batch size.

In [106]:
#Training

from random import sample
import tqdm
import os
import subprocess
import random

def shuffle_sentences(sentences, tags):
    shuffled_sentences = []
    shuffled_tags      = []
    indices = list(range(len(sentences)))
    random.shuffle(indices)
    for i in indices:
        shuffled_sentences.append(sentences[i])
        shuffled_tags.append(tags[i])
    return (shuffled_sentences, shuffled_tags)

nEpochs = 100

def train_basic_lstm(sentences, tags, lstm, glove):
  #TODO: initialize optimizer
    #loss_function = nn.NLLLoss()
    with torch.no_grad():
      lstm.init_glove(glove)

    optimizer = optim.Adadelta(lstm.parameters(),lr=0.5) #lr=0.1)
    #optimizer = optim.SGD(lstm.parameters(),lr=0.1)
    batchSize = 50

    for epoch in range(nEpochs):
        totalLoss = 0.0

        (sentences_shuffled, tags_shuffled) = shuffle_sentences(sentences, tags)
        if lstm.debug:
          print('sentences shuffled: ' + str(len(sentences_shuffled)))
          print('tags shuffled: ' + str(len(tags_shuffled)))
        for batch in tqdm.notebook.tqdm(range(0, len(sentences), batchSize), leave=False):
            lstm.zero_grad()
            #TODO: Implement gradient update.
              # https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

            X_batch = sentences2indices(sentences_shuffled[batch:batch+batchSize], word2i, train=True) #train=True
            Y_batch = sentences2indices(tags_shuffled[batch:batch+batchSize], tag2i)
            if lstm.debug:
              print('len(X_batch): ' + str(len(X_batch)))

            X_batch_prepared = prepare_input(X_batch)[0].cuda() # [0] returns padded sequence, [1] returns mask
            Y_batch_onehot   = prepare_output_onehot(Y_batch).cuda() #.argmax(dim=1)
            if lstm.debug:
              print('X_batch_prepared: ' + str(X_batch_prepared.shape))
              print('Y_onehot: ' + str(Y_batch_onehot.shape))

            word_embeds = lstm.word_embeddings(X_batch_prepared)
            if lstm.debug:
              print('word_embeds: ' + str(word_embeds.shape))
            pred = lstm.forward(word_embeds)
            if lstm.debug:
              print('pred: ' + str(pred.shape))
            
            #loss = loss_function(pred, Y_batch_onehot)
            loss = torch.einsum('bij,bij',torch.neg(pred),Y_batch_onehot) / batchSize
            loss.backward()
            optimizer.step()
            totalLoss += loss

        print(f"loss on epoch {epoch} = {totalLoss}")
        lstm.write_predictions(sentences_dev, 'dev_pred')   #Performance on dev set
        print('conlleval:')
        print(subprocess.Popen('paste dev dev_pred | perl conlleval.pl -d "\t"', shell=True, stdout=subprocess.PIPE,stderr=subprocess.STDOUT).communicate()[0].decode('UTF-8'))

        if epoch % 10 == 0:
            s = sample(range(len(sentences_dev)), 5)
            lstm.print_predictions([sentences_dev[i] for i in s], [tags_dev[i] for i in s])

torch.manual_seed(2)

lstm = BasicLSTMtagger(DIM_HID=500, DIM_EMB=300, debug=False).cuda()
train_basic_lstm(sentences_train, tags_train, lstm, GloVe)

  0%|          | 0/300 [00:00<?, ?it/s]

loss on epoch 0 = 871.011474609375
conlleval:
processed 51578 tokens with 5942 phrases; found: 6081 phrases; correct: 4922.
accuracy:  97.05%; precision:  80.94%; recall:  82.83%; FB1:  81.88
              LOC: precision:  90.16%; recall:  86.34%; FB1:  88.21  1759
             MISC: precision:  78.01%; recall:  73.10%; FB1:  75.48  864
              ORG: precision:  64.40%; recall:  75.69%; FB1:  69.59  1576
              PER: precision:  87.51%; recall:  89.41%; FB1:  88.45  1882

----------------------------
-START-/START/START Chinese/I-MISC/I-MISC police/O/O have/O/O detained/O/O veteran/O/O dissident/O/O Wang/I-PER/I-PER Donghai/I-PER/I-PER ,/O/O the/O/O New/I-ORG/I-MISC York-based/I-ORG/I-MISC pressure/O/O group/O/O Human/I-ORG/I-ORG Rights/I-ORG/I-ORG in/O/I-ORG China/I-LOC/I-ORG said/O/O on/O/O Saturday/O/O ./O/O -END-/END/END
Predicted:	 ['START', 'I-MISC', 'O', 'O', 'O', 'O', 'O', 'I-PER', 'I-PER', 'O', 'O', 'I-ORG', 'I-ORG', 'O', 'O', 'I-ORG', 'I-ORG', 'O', 'I-LOC', 'O', 'O

  0%|          | 0/300 [00:00<?, ?it/s]

loss on epoch 1 = 334.00885009765625
conlleval:
processed 51578 tokens with 5942 phrases; found: 5861 phrases; correct: 5063.
accuracy:  97.75%; precision:  86.38%; recall:  85.21%; FB1:  85.79
              LOC: precision:  91.36%; recall:  89.82%; FB1:  90.58  1806
             MISC: precision:  80.17%; recall:  81.13%; FB1:  80.65  933
              ORG: precision:  77.54%; recall:  74.65%; FB1:  76.06  1291
              PER: precision:  90.88%; recall:  90.34%; FB1:  90.61  1831



  0%|          | 0/300 [00:00<?, ?it/s]

...

loss on epoch 97 = 0.8994585275650024
conlleval:
processed 51578 tokens with 5942 phrases; found: 6045 phrases; correct: 5348.
accuracy:  98.30%; precision:  88.47%; recall:  90.00%; FB1:  89.23
              LOC: precision:  94.18%; recall:  92.43%; FB1:  93.30  1803
             MISC: precision:  82.83%; recall:  83.19%; FB1:  83.01  926
              ORG: precision:  78.26%; recall:  85.38%; FB1:  81.67  1463
              PER: precision:  93.79%; recall:  94.35%; FB1:  94.07  1853



  0%|          | 0/300 [00:00<?, ?it/s]

loss on epoch 98 = 0.4565410614013672
conlleval:
processed 51578 tokens with 5942 phrases; found: 5991 phrases; correct: 5345.
accuracy:  98.40%; precision:  89.22%; recall:  89.95%; FB1:  89.58
              LOC: precision:  94.40%; recall:  92.65%; FB1:  93.52  1803
             MISC: precision:  83.59%; recall:  83.41%; FB1:  83.50  920
              ORG: precision:  80.75%; recall:  85.38%; FB1:  83.00  1418
              PER: precision:  93.46%; recall:  93.87%; FB1:  93.66  1850



  0%|          | 0/300 [00:00<?, ?it/s]

loss on epoch 99 = 0.8050068020820618
conlleval:
processed 51578 tokens with 5942 phrases; found: 5978 phrases; correct: 5346.
accuracy:  98.41%; precision:  89.43%; recall:  89.97%; FB1:  89.70
              LOC: precision:  94.77%; recall:  92.65%; FB1:  93.70  1796
             MISC: precision:  83.24%; recall:  83.51%; FB1:  83.38  925
              ORG: precision:  81.55%; recall:  85.38%; FB1:  83.42  1404
              PER: precision:  93.31%; recall:  93.87%; FB1:  93.59  1853



In [107]:
#Evaluation on test data
lstm.write_predictions(sentences_test, 'test_pred_lstm.txt')
!wget https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl
!paste test test_pred_lstm.txt | perl conlleval.pl -d "\t"

--2023-03-24 03:15:40--  https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12754 (12K) [text/plain]
Saving to: ‘conlleval.pl.8’

conlleval.pl.8      100%[===================>]  12.46K  --.-KB/s    in 0s      

2023-03-24 03:15:40 (88.5 MB/s) - ‘conlleval.pl.8’ saved [12754/12754]

processed 46666 tokens with 5648 phrases; found: 5695 phrases; correct: 4791.
accuracy:  97.08%; precision:  84.13%; recall:  84.83%; FB1:  84.48
              LOC: precision:  89.89%; recall:  86.39%; FB1:  88.11  1603
             MISC: precision:  69.95%; recall:  74.93%; FB1:  72.35  752
              ORG: precision:  78.18%; recall:  82.84%; FB1:  80.44  1760
              PER: precis

## Initialization with GloVe Embeddings (5 points)

If you haven't already, implement the `init_glove()` method in `BasicLSTMtagger` above.

Rather than initializing word embeddings randomly, it is common to use learned word embeddings (GloVe or Word2Vec), as discussed in lecture.  To make this simpler, we have already pre-filtered [GloVe](https://nlp.stanford.edu/projects/glove/) embeddings to only contain words in the vocabulary of the CoNLL NER dataset, and loaded them into a dictionary (`GloVe`) at the beginning of this notebook.



## Character Embeddings (10 points)

Now that you have your basic LSTM tagger working, the next step is to add a convolutional network that computes word embeddings from character representations of words.  See Figure 2 and Figure 3 in the [Ma and Hovy](https://www.aclweb.org/anthology/P16-1101.pdf) paper.  We have provided code in `sentences2input_tensors` to convert sentences into lists of word and character indices.  See also [nn.Conv1d](https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html) and [MaxPool1d](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool1d.html).

Hint: The nn.Conv1d accepts input size $(N, C_{in}, L_{in})$, but we have input size $(N, \text{SLEN}, \text{CLEN}, \text{EMB_DIM})$. We can reshape and [permute](https://pytorch.org/docs/stable/generated/torch.permute.html) our input to satisfy the nn.Conv1d, and recover the dimensions later.

Make sure to save your predictions on the test set, for submission to GradeScope.  You should be able to achieve **90 F1 / 85 F1 on the dev/test sets**.

In [108]:
import torch.nn.functional as F


class CharLSTMtagger(BasicLSTMtagger): 
    def __init__(self, DIM_EMB=10, DIM_CHAR_EMB=30, DIM_HID=10, VOCAB_SIZE=29148, debug=True):
        super(CharLSTMtagger, self).__init__(DIM_EMB=DIM_EMB, DIM_HID=DIM_HID, VOCAB_SIZE=VOCAB_SIZE, debug=debug)
        NUM_TAGS = max(tag2i.values())+1

        (self.DIM_EMB, self.DIM_CHAR_EMB, self.NUM_TAGS) = (DIM_EMB, DIM_CHAR_EMB, NUM_TAGS)

        #TODO: Initialize parameters.
        self.char_embeddings = nn.Embedding(num_embeddings=VOCAB_SIZE, embedding_dim=DIM_CHAR_EMB)
        self.cnn = nn.Conv1d(in_channels=DIM_CHAR_EMB, out_channels=DIM_CHAR_EMB, kernel_size=3)
        self.max_pool = nn.MaxPool1d(kernel_size=DIM_CHAR_EMB) # 1 = floor((30+2*padding - dilation*(kernel_size-1)-1)/stride + 1) ---> 30+2*padding = dilation*(kernel_size-1) + 1 ---> kernel_size=30

        self.lstm = nn.LSTM(input_size=DIM_EMB+DIM_CHAR_EMB, hidden_size=DIM_HID, batch_first=True, bidirectional=True)
        self.drop = nn.Dropout(p=0.2)

        self.debug = debug
        if self.debug:
          print('DIM_CHAR_EMB: ' + str(DIM_CHAR_EMB) + '\n')

    def forward(self, X, X_char, train=False):
        #TODO: Implement the forward computation.
        BATCH_SIZE, SLEN, CLEN = X_char.shape

        ## generate word embeddings - MAKE SURE WE'VE INITIALIZED GLOVE EMBEDDINGS BEFORE CALLING FORWARD!
        word_embeds = self.word_embeddings(X)
        if self.debug:
          print('word_embeds: ' + str(word_embeds.shape) + '\n')

        ## generate char embeddings
        char_embeds = self.char_embeddings(X_char)
        if self.debug:
          print('char_embeds: ' + str(char_embeds.shape))

        permuted_char_embeds = torch.permute(char_embeds,(0,1,3,2))
        if self.debug:
          print('permuted_char_embeds: ' + str(permuted_char_embeds.shape))
        
        reshaped_char_embeds = torch.reshape(permuted_char_embeds,(BATCH_SIZE*SLEN,self.DIM_CHAR_EMB,CLEN))
        if self.debug:
          print('reshaped_char_embeds: ' + str(reshaped_char_embeds.shape))

        cnn_char_embeds = self.drop(self.cnn(reshaped_char_embeds))
        if self.debug:
          print('cnn_char_embeds: ' + str(cnn_char_embeds.shape))
        
        max_pool_char_embeds = self.max_pool(cnn_char_embeds)
        if self.debug:
          print('max_pool_char_embeds: ' + str(max_pool_char_embeds.shape))

        reshaped_char_embeds = torch.reshape(max_pool_char_embeds,(BATCH_SIZE,SLEN,self.DIM_CHAR_EMB))
        if self.debug:
          print('reshaped_max_pool_output: ' + str(reshaped_char_embeds.shape) + '\n')

        ## concatenate word and character embeddings
        lstm_input = self.drop(torch.cat((word_embeds, reshaped_char_embeds), dim=2))
        if self.debug:
          print('lstm_input: ' + str(lstm_input.shape))

        ## perform rest of LSTM forward pass as normal
        tag_scores = super(CharLSTMtagger,self).forward(lstm_input)
        return tag_scores
        #pass
        # #return torch.randn((X.shape[0], X.shape[1], self.NUM_TAGS))  #Random baseline.

    def sentences2input_tensors(self, sentences):
        (X, X_mask)   = prepare_input(sentences2indices(sentences, word2i))
        X_char        = prepare_input_char(sentences2indicesChar(sentences, char2i))
        return (X, X_mask, X_char)

    def inference(self, sentences):
        (X, X_mask, X_char) = self.sentences2input_tensors(sentences)
        pred = self.forward(X.cuda(), X_char.cuda()).argmax(dim=2)
        return [[i2tag[pred[i,j].item()] for j in range(len(sentences[i]))] for i in range(len(sentences))]

    def print_predictions(self, words, tags):
        Y_pred = self.inference(words)
        for i in range(len(words)):
            print("----------------------------")
            print(" ".join([f"{words[i][j]}/{Y_pred[i][j]}/{tags[i][j]}" for j in range(len(words[i]))]))
            print("Predicted:\t", Y_pred[i])
            print("Gold:\t\t", tags[i])

char_lstm_test = CharLSTMtagger(DIM_HID=7, DIM_EMB=300)
lstm_output    = char_lstm_test.forward(prepare_input(X[0:5])[0], prepare_input_char(X_char[0:5]))
Y_onehot       = prepare_output_onehot(Y[0:5])

print("lstm output shape:", lstm_output.shape)
print("Y onehot shape:", Y_onehot.shape)

VOCAB_SIZE: 29148
NUM_TAGS: 10
DIM_EMB: 300
DIM_HID: 7
bidirectional?: True

DIM_CHAR_EMB: 30

word_embeds: torch.Size([5, 32, 300])

char_embeds: torch.Size([5, 32, 32, 30])
permuted_char_embeds: torch.Size([5, 32, 30, 32])
reshaped_char_embeds: torch.Size([160, 30, 32])
cnn_char_embeds: torch.Size([160, 30, 30])
max_pool_char_embeds: torch.Size([160, 30, 1])
reshaped_max_pool_output: torch.Size([5, 32, 30])

lstm_input: torch.Size([5, 32, 330])
lstm_out: torch.Size([5, 32, 14])
tag_space: torch.Size([5, 32, 10])
tag_scores: torch.Size([5, 32, 10])

lstm output shape: torch.Size([5, 32, 10])
Y onehot shape: torch.Size([5, 32, 10])


In [109]:
#Training LSTM w/ character embeddings. Feel free to change number of epochs, optimizer, learning rate and batch size.

nEpochs = 100

def train_char_lstm(sentences, tags, lstm, glove):
  #optimizer = optim.Adadelta(lstm.parameters(), lr=0.1)
  #TODO: initialize optimizer
    with torch.no_grad():
      lstm.init_glove(glove)

    optimizer = optim.Adadelta(lstm.parameters(),lr=0.1)

    batchSize = 50

    for epoch in range(nEpochs):
        totalLoss = 0.0

        (sentences_shuffled, tags_shuffled) = shuffle_sentences(sentences, tags)
        for batch in tqdm.notebook.tqdm(range(0, len(sentences), batchSize), leave=False):
            lstm.zero_grad()
            #TODO: Gradient update
            (X_batch_prepared, X_mask_batch_prepared, X_char_batch_prepared) = lstm.sentences2input_tensors(sentences_shuffled[batch:batch+batchSize])
            Y_batch_onehot   = prepare_output_onehot(sentences2indices(tags_shuffled[batch:batch+batchSize], tag2i)).cuda()
            if lstm.debug:
              print('X_batch_prepared: ' + str(X_batch_prepared.shape))
              print('X_char_batch_prepared: ' + str(X_char_batch_prepared.shape))
              print('Y_onehot: ' + str(Y_batch_onehot.shape))

            pred = lstm.forward(X_batch_prepared.cuda() ,X_char_batch_prepared.cuda()) # TODO
            if lstm.debug:
              print('pred: ' + str(pred.shape))
            
            #loss = loss_function(pred, Y_batch_onehot)
            loss = torch.einsum('bij,bij',torch.neg(pred),Y_batch_onehot) / batchSize
            loss.backward()
            optimizer.step()
            totalLoss += loss

        print(f"loss on epoch {epoch} = {totalLoss}")
        lstm.write_predictions(sentences_dev, 'dev_pred')   #Performance on dev set
        print('conlleval:')
        print(subprocess.Popen('paste dev dev_pred | perl conlleval.pl -d "\t"', shell=True, stdout=subprocess.PIPE,stderr=subprocess.STDOUT).communicate()[0].decode('UTF-8'))

        if epoch % 10 == 0:
            s = sample(range(len(sentences_dev)), 5)
            lstm.print_predictions([sentences_dev[i] for i in s], [tags_dev[i] for i in s])

torch.manual_seed(1)

char_lstm = CharLSTMtagger(DIM_HID=500, DIM_EMB=300, debug=False).cuda()
train_char_lstm(sentences_train, tags_train, char_lstm, GloVe)

  0%|          | 0/300 [00:00<?, ?it/s]

loss on epoch 0 = 1489.671142578125
conlleval:
processed 51578 tokens with 5942 phrases; found: 5882 phrases; correct: 4361.
accuracy:  95.62%; precision:  74.14%; recall:  73.39%; FB1:  73.77
              LOC: precision:  85.01%; recall:  79.64%; FB1:  82.24  1721
             MISC: precision:  67.68%; recall:  62.69%; FB1:  65.09  854
              ORG: precision:  57.43%; recall:  54.74%; FB1:  56.05  1278
              PER: precision:  78.17%; recall:  86.10%; FB1:  81.94  2029

----------------------------
-START-/START/START Wrexham/I-ORG/I-ORG 2/O/O 0/O/O 2/O/O 0/O/O 5/O/O 5/O/O 2/O/O -END-/END/END
Predicted:	 ['START', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'END']
Gold:		 ['START', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'END']
----------------------------
-START-/START/START CHISINAU/I-LOC/I-LOC ,/O/O Moldova/I-LOC/I-LOC 1996-08-31/O/O -END-/END/END
Predicted:	 ['START', 'I-LOC', 'O', 'I-LOC', 'O', 'END']
Gold:		 ['START', 'I-LOC', 'O', 'I-LOC', 'O', 'END']
---------

  0%|          | 0/300 [00:00<?, ?it/s]

loss on epoch 1 = 533.2505493164062
conlleval:
processed 51578 tokens with 5942 phrases; found: 6343 phrases; correct: 4754.
accuracy:  96.28%; precision:  74.95%; recall:  80.01%; FB1:  77.40
              LOC: precision:  88.52%; recall:  79.37%; FB1:  83.70  1647
             MISC: precision:  73.02%; recall:  68.11%; FB1:  70.48  860
              ORG: precision:  53.63%; recall:  76.66%; FB1:  63.11  1917
              PER: precision:  85.46%; recall:  89.03%; FB1:  87.21  1919



  0%|          | 0/300 [00:00<?, ?it/s]

...

loss on epoch 97 = 0.3950183093547821
conlleval:
processed 51578 tokens with 5942 phrases; found: 6087 phrases; correct: 5250.
accuracy:  98.10%; precision:  86.25%; recall:  88.35%; FB1:  87.29
              LOC: precision:  90.51%; recall:  90.36%; FB1:  90.44  1834
             MISC: precision:  80.30%; recall:  81.78%; FB1:  81.03  939
              ORG: precision:  78.00%; recall:  83.00%; FB1:  80.42  1427
              PER: precision:  91.31%; recall:  93.54%; FB1:  92.41  1887



  0%|          | 0/300 [00:00<?, ?it/s]

loss on epoch 98 = 0.4104381203651428
conlleval:
processed 51578 tokens with 5942 phrases; found: 6110 phrases; correct: 5249.
accuracy:  98.06%; precision:  85.91%; recall:  88.34%; FB1:  87.11
              LOC: precision:  90.70%; recall:  89.77%; FB1:  90.23  1818
             MISC: precision:  81.05%; recall:  82.10%; FB1:  81.57  934
              ORG: precision:  75.45%; recall:  84.12%; FB1:  79.55  1495
              PER: precision:  92.06%; recall:  93.11%; FB1:  92.58  1863



  0%|          | 0/300 [00:00<?, ?it/s]

loss on epoch 99 = 0.5043244957923889
conlleval:
processed 51578 tokens with 5942 phrases; found: 6102 phrases; correct: 5254.
accuracy:  98.12%; precision:  86.10%; recall:  88.42%; FB1:  87.25
              LOC: precision:  90.52%; recall:  90.47%; FB1:  90.50  1836
             MISC: precision:  79.94%; recall:  81.24%; FB1:  80.58  937
              ORG: precision:  77.00%; recall:  83.89%; FB1:  80.30  1461
              PER: precision:  91.97%; recall:  93.27%; FB1:  92.61  1868



In [110]:
#Evaluation on test set
char_lstm.write_predictions(sentences_test, 'test_pred_cnn_lstm.txt')
!wget https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl
!paste test test_pred_cnn_lstm.txt | perl conlleval.pl -d "\t"

--2023-03-24 03:47:16--  https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12754 (12K) [text/plain]
Saving to: ‘conlleval.pl.9’

conlleval.pl.9      100%[===================>]  12.46K  --.-KB/s    in 0s      

2023-03-24 03:47:16 (121 MB/s) - ‘conlleval.pl.9’ saved [12754/12754]

processed 46666 tokens with 5648 phrases; found: 5936 phrases; correct: 4709.
accuracy:  96.62%; precision:  79.33%; recall:  83.37%; FB1:  81.30
              LOC: precision:  84.46%; recall:  83.39%; FB1:  83.92  1647
             MISC: precision:  68.30%; recall:  73.36%; FB1:  70.74  754
              ORG: precision:  70.16%; recall:  81.10%; FB1:  75.23  1920
              PER: precisi

## Conditional Random Fields (15 points)

Now we are ready to add a CRF layer to the `CharacterLSTMTagger`.  To train the model, implement `conditional_log_likelihood`, using the score (unnormalized log probability) of the gold sequence, in addition to the partition function, $Z(X)$, which is computed using the forward algorithm.  Then, you can simply use Pytorch's automatic differentiation to compute gradients by running backpropagation through the computation graph of the dynamic program (this should be very simple, so long as you are able to correctly implement the forward algorithm using a computation graph that is supported by PyTorch).  This approach to computing gradients for CRFs is discussed in Section 7.5.3 of the [Eisenstein Book](https://github.com/jacobeisenstein/gt-nlp-class/blob/master/notes/eisenstein-nlp-notes.pdf)

You will also need to implement the Viterbi algorithm for inference during decoding.

After including CRF training and Viterbi decoding, you should be getting about **92 F1 / 88 F1 on the dev and test set**, respectively.

**IMPORTANT:** Note that training will be substantially slower this time - depending on the efficiency of your implementation, it could take about 5 minutes per epoch (e.g. 50 minutes for 10 iterations).  It is recommended to start out training on a single batch of data (and testing on this same batch), so that you can quickly debug, making sure your model can memorize the labels on a single batch, and then optimize your code.  Once you are fairly confident your code is working properly, then you can train using the full dataset.  We have provided a (commented out) line of code to switch between training on a single batch and the full dataset below.

**Hint #1:** While debugging your implementation of the Forward algorithm it is helpful to look at the loss during training.  The loss should never be less than zero (the log-likelihood should always be negative).

**Hint #2:** To sum log-probabilities in a numerically stable way at the end of the Forward algorithm, you will want to use [`torch.logsumexp`](https://pytorch.org/docs/stable/generated/torch.logsumexp.html).

In [ ]:
#For F.max_pool1d()
import torch.nn.functional as F

class LSTM_CRFtagger(CharLSTMtagger):
    def __init__(self, DIM_EMB=10, DIM_CHAR_EMB=30, DIM_HID=10, N_TAGS=max(tag2i.values())+1):
        super(LSTM_CRFtagger, self).__init__(DIM_EMB=DIM_EMB, DIM_HID=DIM_HID, DIM_CHAR_EMB=DIM_CHAR_EMB)

        #TODO: Initialize parameters.

        self.transitionWeights = nn.Parameter(torch.zeros((N_TAGS, N_TAGS), requires_grad=True))
        nn.init.normal_(self.transitionWeights)

    def gold_score(self, lstm_scores, Y):
        #TODO: compute score of gold sequence Y (unnormalized conditional log-probability)
        return 0

    #Forward algorithm for a single sentence
    #Efficiency will eventually be important here.  We recommend you start by 
    #training on a single batch and make sure your code can memorize the 
    #training data.  Then you can go back and re-write the inner loop using 
    #tensor operations to speed things up.
    def forward_algorithm(self, lstm_scores, sLen):
        #TODO: implement forward algorithm.
        return 0

    def conditional_log_likelihood(self, sentences, tags, train=True):
        #Todo: compute conditional log likelihood of Y (use forward_algorithm and gold_score)
        return 0

    def viterbi(self, lstm_scores, sLen):
        #TODO: Implement Viterbi algorithm, soring backpointers to recover the argmax sequence.  Returns the argmax sequence in addition to its unnormalized conditional log-likelihood.
        return (torch.as_tensor([random.randint(0,lstm_scores.shape[1]-1) for x in range(sLen)]), 0)

    #Computes Viterbi sequences on a batch of data.
    def viterbi_batch(self, sentences):
        viterbiSeqs = []
        (X, X_mask, X_char) = self.sentences2input_tensors(sentences)
        lstm_scores = self.forward(X.cuda(), X_char.cuda())
        for s in range(len(sentences)):
            (viterbiSeq, ll) = self.viterbi(lstm_scores[s], len(sentences[s]))
            viterbiSeqs.append(viterbiSeq)
        return viterbiSeqs

    def forward(self, X, X_char, train=False):
        #TODO: Implement the forward computation.
        return torch.randn((X.shape[0], X.shape[1], self.NUM_TAGS))  #Random baseline.

    def print_predictions(self, words, tags):
        Y_pred = self.inference(words)
        for i in range(len(words)):
            print("----------------------------")
            print(" ".join([f"{words[i][j]}/{Y_pred[i][j]}/{tags[i][j]}" for j in range(len(words[i]))]))
            print("Predicted:\t", [Y_pred[i][j] for j in range(len(words[i]))])
            print("Gold:\t\t", tags[i])

    #Need to use Viterbi this time.
    def inference(self, sentences, viterbi=True):
        pred = self.viterbi_batch(sentences)
        return [[i2tag[pred[i][j].item()] for j in range(len(sentences[i]))] for i in range(len(sentences))]

lstm_crf = LSTM_CRFtagger(DIM_EMB=300).cuda()

In [ ]:
print(lstm_crf.conditional_log_likelihood(sentences_dev[0:3], tags_dev[0:3]))

In [ ]:
#CharLSTM-CRF Training. Feel free to change number of epochs, optimizer, learning rate and batch size.
import tqdm
import os
import subprocess
import random

nEpochs = 10

#Get CoNLL evaluation script
os.system('wget https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl')

def train_crf_lstm(sentences, tags, lstm):
    #optimizer = optim.Adadelta(lstm.parameters(), lr=1.0)
    #TODO: initialize optimizer

    batchSize = 50

    for epoch in range(nEpochs):
        totalLoss = 0.0
        lstm.train()

        #Shuffle the sentences
        (sentences_shuffled, tags_shuffled) = shuffle_sentences(sentences, tags)
        for batch in tqdm.notebook.tqdm(range(0, len(sentences), batchSize), leave=False):
            lstm.zero_grad()
            #TODO: take gradient step on a batch of data.

        print(f"loss on epoch {epoch} = {totalLoss}")
        lstm.write_predictions(sentences_dev, 'dev_pred')   #Performance on dev set
        print('conlleval:')
        print(subprocess.Popen('paste dev dev_pred | perl conlleval.pl -d "\t"', shell=True, stdout=subprocess.PIPE,stderr=subprocess.STDOUT).communicate()[0].decode('UTF-8'))

        if epoch % 10 == 0:
            lstm.eval()
            s = random.sample(range(50), 5)
            lstm.print_predictions([sentences_train[i] for i in s], [tags_train[i] for i in s])   #Print predictions on train data (useful for debugging)

crf_lstm = LSTM_CRFtagger(DIM_HID=500, DIM_EMB=300, DIM_CHAR_EMB=30).cuda()
train_crf_lstm(sentences_train, tags_train, crf_lstm)             #Train on the full dataset
#train_crf_lstm(sentences_train[0:50], tags_train[0:50])          #Train only the first batch (use this during development/debugging)

In [ ]:
crf_lstm.eval()
crf_lstm.write_predictions(sentences_test, 'test_pred_cnn_lstm_crf.txt')
!wget https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl
!paste test test_pred_cnn_lstm_crf.txt | perl conlleval.pl -d "\t"

## Gradescope

Gradescope allows you to add multiple files to your submission. Please submit this notebook along with the test set prediction:
* test_pred_lstm.txt
* test_pred_cnn_lstm.txt
* test_pred_cnn_lstm_crf.txt
* NER_release.ipynb

To download this notebook, go to `File > Download.ipynb`. You can download the predictions from Colab by clicking the folder icon on the left and finding them under Files. 

Please make sure that you name the files as specified above. You will be able to see the test set accuracy for your predictions. However, the final score will be assigned later based on accuracy and implementation. 

When submitting the .ipynb notebook, please make sure that all the cells run when executed in order starting from a fresh session. If the code doesn't take too long to run, you can re-run everything with `Runtime -> Restart and run all`

You can submit multiple times before the deadline and choose the submission which you want to be graded by going to `Submission History` on gradescope.
